In [10]:
#Initialization for HateSpeechRecognition
import scipy as sp
import numpy as np
import pandas as pd
import sklearn
from sklearn import svm
import nltk
import re
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import zero_one_loss
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from scipy.stats import t
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/mufa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mufa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mufa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [11]:
def nested_kfold_cv(clf, param_grid, X, y, k1=10, k2=3):
    
    err = []
    kfold = KFold(n_splits=k1, shuffle=True, random_state=42)
    
    # Outer loop
    for ind_train, ind_test in kfold.split(X):
        
        X_train, y_train, X_test, y_test = X[ind_train], y[ind_train], X[ind_test], y[ind_test]
        
        # Inner loop
        inn = GridSearchCV(clf,param_grid,n_jobs=5, cv=StratifiedKFold(n_splits=5, 
                                              random_state=42).split(X_train, y_train), 
                           verbose=2).fit(X_train, y_train)
        
        # Prediction based on the best selected params, the ones that minimize average error
        h = inn.best_estimator_.fit(X_train, y_train).predict(X_test)
        
        err.append(zero_one_loss(y_test, h))
        print(classification_report( y_test, h ))
    return err

In [12]:

def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, '', new_text)
    return new_text

In [13]:

stopwords_list = stopwords.words('english')
#other_exclusions = ["#ff", "ff", "rt","!",":","...",".","-","&","?"]
#stopwords_list.extend(other_exclusions)

dataset = pd.read_csv("dataset/labeled_data.csv")

tweets = dataset.tweet

ps = nltk.stem.PorterStemmer()

#ps.stem(dataset)

#Text cleaning and tokenization, then stemming then POS tagging
filtered_tweets=[];
tweet_tags = [];
filtered_tweets_stemmed=[];

for tweet in tweets:
    tweet = clean_text(["http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",".*@.*:","&#*\w*","@[\w\-]+","[^\w\s]"],tweet)
    tweet = tweet.lower()
    word_tokens = word_tokenize(tweet)
    filtered_tweets.append([word for word in word_tokens if not word in stopwords_list])
    filtered_tweets_stemmed.append([ps.stem(word) for word in word_tokens if not word in stopwords_list])
    
for tweet in filtered_tweets:
    tweet_tags.append(nltk.pos_tag(tweet))

    

In [14]:
#Creating TfIdf vectorizer
ftss=[]
for tweet in filtered_tweets_stemmed:
    ftss.append(' '.join(tweet))

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),lowercase=False,max_features=1000,smooth_idf=False,norm=None,max_df=0.75,min_df=5)



In [15]:
X = vectorizer.fit_transform(ftss).toarray()
Y = dataset['class']

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.3)



In [ ]:
#SVM Classifier
C = [ 2e-2, 2e-1, 2e-0]
gama = [ 2e-2, 2e-1, 2e-0]
param = [{'svc__kernel': ['rbf'], 'svc__gamma': C, 'svc__C': gama}]
clf = svm.SVC(decision_function_shape='ovo')
scale = StandardScaler()
pipeline = Pipeline([('scaler', scale), ('svc', clf)])

svm_err = nested_kfold_cv(pipeline, param, X, Y)
#param_grid = [{}] 
#grid_search = GridSearchCV(pipeline, 
#                           param_grid,
#                          n_jobs = 5,
#                           cv=StratifiedKFold(n_splits=5, 
#                                              random_state=42).split(X_train, y_train), 
#                           verbose=2)
#model = grid_search.fit(X_train,y_train)
#y_pred = model.predict(X_test)
#report = classification_report( y_test, y_pred )
#print(report)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] svc__C=0.02, svc__gamma=0.02, svc__kernel=rbf ...................
[CV] svc__C=0.02, svc__gamma=0.02, svc__kernel=rbf ...................
[CV] svc__C=0.02, svc__gamma=0.02, svc__kernel=rbf ...................
[CV] svc__C=0.02, svc__gamma=0.02, svc__kernel=rbf ...................
[CV] svc__C=0.02, svc__gamma=0.02, svc__kernel=rbf ...................


In [15]:
#Logistic Regression

C = [2e-3, 2e-2, 2e-1, 2e-0, 2e-1, 2e-2, 2e-3]

std_scaler = StandardScaler()
clf = LogisticRegression(multi_class='ovr',solver='newton-cg')
pipeline = Pipeline([('scaler', std_scaler)  , ('clf', clf)])
param = [{'clf__C': C}]

logreg_err = nested_kfold_cv(pipeline, param, X, Y)



#TODO: logicstic regression, model selection, kfold

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................


MemoryError: 

In [ ]:
txt, t_stat, txt, p_val = paired_t_test(logreg_err, svm_err)

if p_val <= 0.05:
    print '\np-value = ', p_val, ', hipoteza se odbacuje.'
else:
    print '\np-value = ', p_val, ', hipoteza se ne odbacuje.'